# Introduction to CARDAMOM runs: running a local assimilation and forward run
This tutorial will show the basic set up to run a CARDAMOM assimilation and forward run from the command line. This is set up for mac/linux.

## Basic Steps:
1. Download/git-clone a copy of CARDAMOM
2. Compile the c-code of CARDAMOM (make sure to do this after any changes)
3. Run compiled file for assimilation with a pointer towards the input (.cbf) and parameters (.cbr) locations.
4. With a parameters file, run a compiled file for a forward run to retrieve the outputs of your parameter ensemble. Point required for outputs (.bin).
5. Common practice is to organize input, parameter and output files in:
    topfolder/cbf/*.cbf
    topfolder/cbr/*.cbr
    topfolder/output/*.bin

### Folder structure for the tutorial

In [6]:
# Other import here for name lists?
import os
import sys
import subprocess
MACHDIR = os.path.expanduser("~")

dir_script = '../cardamom_utilities/readwritebinary'
sys.path.append(dir_script)



# Location of the local CARDAMOM repository
# Location of git repository: email: gquetin@stanford.edu
dircardamom = '/Users/gquetin/repos/models/cardamom_share/' #<< update this for your CARDAMOM location

# define the directory where the test data is (require someone to input their home directory)
dirbinary = './data/inputoutput_point/'

# By convention, CARDAMOM outputs are separate into a cbf (input) directory, 
# a cbr (parameters) directory, and an output directory
dir_cbf = dirbinary + 'cbf/'
dir_cbr = dirbinary + 'cbr/'
dir_output = dirbinary + 'output/'

### Compile CARDAMOM

In [5]:
# Compile code
# Can run something like these in command line:
# gcc -std=c99 CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.c -o CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.exe -lm
# gcc -std=c99 CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.c -o CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.exe -lm
# Or create .bash file similar to: "CARDAMOM_compile.sh"

# For the forward run executable
exitcode_forward = subprocess.run(["gcc", 
                                   "-std=c99",
                                   dircardamom + "CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.c",
                                   "-o",
                                   dircardamom + "CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.exe", 
                                   "-lm"])

exitcode_assim = subprocess.run(["gcc", 
                                 "-std=c99",
                                 dircardamom + "CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.c",
                                 "-o",
                                 dircardamom + "CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.exe", 
                                 "-lm"])

# As long as the exit code == 0, you now have a compiled version of CARDAMOM ready to run.
print("The exit code was: %d" % exitcode_forward.returncode)
print("The exit code was: %d" % exitcode_assim.returncode)



The exit code was: 0
The exit code was: 0


### Run CARDAMOM Assimilation and Forward Run
This is for running CARDAMOM at one point and locally, <> see Sherlock tutorial for helpful scripts when running many points.

By convention, when running CARDAMOM at multiple spatial points, the 4 digits at the end of the name are reserved for the row and column of thos spatial points on a grid. For point runs leave '0000'. For cbr files the last digit is reserved for the chain number being run

Naming convention for outputs:
1. Fluxes - 'fluxfile'
2. Pools - 'poolfile'
3. EDCD pass/no pass - 'edcdfile'
4. Probabilities from cost function - 'probfile'

In [4]:

executable_assim = dircardamom + "CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.exe"
cardamom_cbf = dir_cbf + 'test_1962.cbf'
cardamom_cbr = dir_cbr + 'test_1962_0.cbr'

# Command line settings
# From command line:
# /Users/gquetin/repos/models/cardamom_share/CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.exe /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/cbf/e1a_USHa1_id821_test_coplai_div100_lwbias100_0000.cbf /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/cbr/e1a_USHa1_id821_test_coplai_div100_lwbias100_testtest_0000.cbr 100000 0 200 0.001 119 1000
number_iterations = "100000" # Small numbers for test, 100000000 standard run, will take a long time.
printrate = "0" #
samplerate = "200" # Number of output parameters = number_iterations/samplerate, generally use between 500 and 1000 output parameters
minimum_step_size = ".001" # Relates to MCMC, standard is currently .001
mcmcid = "119" # Current default MCMC method is 119
nadapt = "1000" # MCMC setting, standard is 1000

# Run assimilation
cmdline_submit_assim = [executable_assim,
                        cardamom_cbf,
                        cardamom_cbr,
                        number_iterations,
                        printrate,
                        samplerate,
                        minimum_step_size,
                        mcmcid,
                        nadapt]


print('Submit for assimilation runs: \n' +
      " ".join(cmdline_submit_assim)+'\n')

# Running from python currently not set up
#exitcode_assim = subprocess.run(cmdline_submit_assim)                               
#print("The exit code was: %d" % exitcode_assim.returncode)                                 
                                 
                                 
# Forward run
# Example for command line
# /Users/gquetin/repos/models/cardamom_share/CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.exe /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/cbf/e1a_USHa1_id821_test_coplai_div100_lwbias100_0000.cbf /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/cbr/e1a_USHa1_id821_test_coplai_div100_lwbias100_0000.cbr /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/output/fluxfile_e1a_USHa1_id821_test_coplai_div100_lwbias100_0000_1.bin /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/output/poolfile_e1a_USHa1_id821_test_coplai_div100_lwbias100_0000_1.bin /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/output/edcdfile_e1a_USHa1_id821_test_coplai_div100_lwbias100_0000_1.bin /Users/gquetin/Google\ Drive/DATA/DALEC/harvard_test/lai_bias/output/probfile_e1a_USHa1_id821_test_coplai_div100_lwbias100_0000_1.bin

# File setup
executable_forward = dircardamom + "CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.exe"

basename = "test"
locpoint = "1962"
chain_number = "0"
cardamom_output_flux = "{}_{}_{}_{}_{}".format(dir_output,'fluxfile',basename,locpoint,chain_number)
cardamom_output_pool = "{}_{}_{}_{}_{}".format(dir_output,'poolfile',basename,locpoint,chain_number)
cardamom_output_edcd = "{}_{}_{}_{}_{}".format(dir_output,'edcdfile',basename,locpoint,chain_number)
cardamom_output_prob = "{}_{}_{}_{}_{}".format(dir_output,'probfile',basename,locpoint,chain_number)


cmdline_submit_forward = [executable_forward,
                          cardamom_cbf,
                          cardamom_cbr,
                          cardamom_output_flux,
                          cardamom_output_pool,
                          cardamom_output_edcd,
                          cardamom_output_prob]

print('Submit for forward runs: \n' +
      " ".join(cmdline_submit_forward))

# Running from python currently not set up
#exitcode_forward = subprocess.run()                              
#print("The exit code was: %d" % exitcode_forward.returncode)   



Submit for assimilation runs: 
/Users/gquetin/repos/models/cardamom_share/CARDAMOM/C/projects/CARDAMOM_MDF/CARDAMOM_MDF.exe ./data/inputoutput_point/cbf/test_1962.cbf ./data/inputoutput_point/cbr/test_1962_0.cbr 100000 0 200 .001 119 1000

Submit for forward runs: 
/Users/gquetin/repos/models/cardamom_share/CARDAMOM/C/projects/CARDAMOM_GENERAL/CARDAMOM_RUN_MODEL.exe ./data/inputoutput_point/cbf/test_1962.cbf ./data/inputoutput_point/cbr/test_1962_0.cbr ./data/inputoutput_point/output/_fluxfile_test_1962_0 ./data/inputoutput_point/output/_poolfile_test_1962_0 ./data/inputoutput_point/output/_edcdfile_test_1962_0 ./data/inputoutput_point/output/_probfile_test_1962_0
